In [2]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [3]:
#Adding rotation to the transforms
# Train Phase transformations
train_transforms2 = transforms.Compose([
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,))
                                       ])

# Test Phase transformations
test_transforms2 = transforms.Compose([
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,))
                                       ])

train = datasets.MNIST('./data', train=True, download=True, transform=train_transforms2)
test = datasets.MNIST('./data', train=False, download=True, transform=test_transforms2)


#setting manual seed
SEED=1

#CUDA?
cuda = torch.cuda.is_available()
print("CUDA Available?", cuda)

#For reproducibility
torch.manual_seed(SEED)

if cuda:
  torch.cuda.manual_seed(SEED)

#dataloader arguments
dataloader_args = dict(shuffle=True, batch_size=128, num_workers=4, pin_memory=True) if cuda else dict(shuffle=True, batch_size=64)

#train dataloader
train_loader = torch.utils.data.DataLoader(train, **dataloader_args)

#test loader
test_loader = torch.utils.data.DataLoader(test, **dataloader_args)

100%|██████████| 9912422/9912422 [00:00<00:00, 99467945.30it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 50684390.72it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw




100%|██████████| 1648877/1648877 [00:00<00:00, 22755181.84it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 7532830.67it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



CUDA Available? True


In [4]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, epoch):
  model.train()
  pbar=tqdm(train_loader)
  correct=0
  processed=0

  for batch_idx, (data, target) in enumerate(pbar):
    #get samples
    data, target = data.to(device), target.to(device)

    #Init
    optimizer.zero_grad()

    #Predict
    y_pred=model(data)

    #Calculate loss
    loss=F.nll_loss(y_pred,target)
    train_losses.append(loss)

    #Backpropagation
    loss.backward()
    optimizer.step()

    #Update pbar-tqdm

    pred = y_pred.argmax(dim=1, keepdim=True) # get the index of the max log-probability
    correct+=pred.eq(target.view_as(pred)).sum().item()
    processed+= len(data)

    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy = {100*correct/processed:0.2f}')
    train_acc.append(100*correct/processed)

def test(model, device, test_loader):
  model.eval()
  test_loss = 0
  correct = 0

  with torch.no_grad():
    for data, target in test_loader:
      data, target = data.to(device), target.to(device)
      output = model(data)
      test_loss += F.nll_loss(output, target, reduction='sum').item() #sum up batch loss
      pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
      correct += pred.eq(target.view_as(pred)).sum().item()

  test_loss /=len(test_loader.dataset)
  test_losses.append(test_loss)

  print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
      test_loss, correct, len(test_loader.dataset),
      100*correct/len(test_loader.dataset)))
  test_acc.append(100* correct/len(test_loader.dataset))

# Model
**Target -**

1. Add Global Average Pooling
2. Add Regularization (Dropouts)
3. Add Global Average Pooling

In [5]:

dropout_value = 0.02
class Net5(nn.Module):
    def __init__(self):
        super(Net5, self).__init__()
        # Input Block
        self.convblock1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(dropout_value)
        ) # output_size = 26 ; RF=3

        # CONVOLUTION BLOCK 1
        self.convblock2 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=16, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(dropout_value)
        ) # output_size = 24 ; RF=5

        # TRANSITION BLOCK 1
        self.convblock3 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=10, kernel_size=(1, 1), padding=0, bias=False),
        ) # output_size = 24 ; RF_in + (k-1)J_in = 5 + (1-1) = 5 ; J_in = 1
        self.pool1 = nn.MaxPool2d(2, 2) # output_size = 12 ; RF=5+(2-1)1 = 6 ; J_out=2

        # CONVOLUTION BLOCK 2
        self.convblock4 = nn.Sequential(
            nn.Conv2d(in_channels=10, out_channels=16, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(dropout_value)
        ) # output_size = 12 ; RF=6+(3-1)*2=10 ; J_Out=2
        
        self.convblock5 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=16, kernel_size=(3, 3), padding=1, bias=False),
            nn.ReLU(),            
            nn.BatchNorm2d(16),
            nn.Dropout(dropout_value)
        ) # output_size = 12 ; RF=10+(3-1)*2=14 ; J_Out=2
        
        # TRANSITION BLOCK 2
        self.pool2 = nn.MaxPool2d(2, 2) # output_size = 6 ; RF=14+(2-1)1 = 15 ; J_out=2
        
        self.convblock6 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=8, kernel_size=(1, 1), padding=0, bias=False),
        ) # output_size = 12 ; RF_in + (k-1)J_in = 14 + (1-1) = 14 ; J_in = 1
       
        
        self.convblock7 = nn.Sequential(
            nn.Conv2d(in_channels=8, out_channels=8, kernel_size=(3, 3), padding=1, bias=False),
            nn.ReLU(),            
            nn.BatchNorm2d(8),
            nn.Dropout(dropout_value)
        ) # output_size = 6 ; RF=10+(3-1)*2=14 ; J_Out=2
        
        self.convblock8 = nn.Sequential(
            nn.Conv2d(in_channels=8, out_channels=8, kernel_size=(3, 3), padding=1, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(8),
            nn.Dropout(dropout_value)
        ) # output_size = 6 ; RF=18+(3-1)*2=22
        
        # OUTPUT BLOCK
        self.gap = nn.Sequential(
            nn.AvgPool2d(kernel_size=5)
        ) # output_size = 1

        self.convblock9 = nn.Sequential(
            nn.Conv2d(in_channels=8, out_channels=10, kernel_size=(1, 1), padding=0, bias=False),
            # nn.BatchNorm2d(10),
            # nn.ReLU(),
            # nn.Dropout(dropout_value)
        )


        self.dropout = nn.Dropout(dropout_value)

    def forward(self, x):
        x = self.convblock1(x)
        x = self.convblock2(x)
        
        x = self.convblock3(x)
        x = self.pool1(x)
        
        x = self.convblock4(x)
        x = self.convblock5(x)
        
        x = self.convblock6(x)
        x = self.pool2(x)
        
        x = self.convblock7(x)
        x = self.convblock8(x)
        x = self.gap(x)        
        x = self.convblock9(x)

        x = x.view(-1,10)
        return F.log_softmax(x, dim=-1)

In [6]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

model = Net5().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             144
              ReLU-2           [-1, 16, 26, 26]               0
       BatchNorm2d-3           [-1, 16, 26, 26]              32
           Dropout-4           [-1, 16, 26, 26]               0
            Conv2d-5           [-1, 16, 24, 24]           2,304
              ReLU-6           [-1, 16, 24, 24]               0
       BatchNorm2d-7           [-1, 16, 24, 24]              32
           Dropout-8           [-1, 16, 24, 24]               0
            Conv2d-9           [-1, 10, 24, 24]             160
        MaxPool2d-10           [-1, 10, 12, 12]               0
           Conv2d-11           [-1, 16, 10, 10]           1,440
             ReLU-12           [-1, 16, 10, 10]               0
      BatchNorm2d-13           [-1, 16, 10, 10]              32
          Dropout-14           [-1, 16,

In [7]:
model = Net5().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
EPOCHS = 15

for epoch in range(EPOCHS):
    print('EPOCH: ',epoch+1)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

EPOCH:  1


Loss=0.14398376643657684 Batch_id=468 Accuracy = 86.39: 100%|██████████| 469/469 [00:08<00:00, 56.59it/s]



Test set: Average loss: 0.1377, Accuracy: 9638/10000 (96.38%)

EPOCH:  2


Loss=0.1405966430902481 Batch_id=468 Accuracy = 97.17: 100%|██████████| 469/469 [00:07<00:00, 60.15it/s]  



Test set: Average loss: 0.0671, Accuracy: 9806/10000 (98.06%)

EPOCH:  3


...


Test set: Average loss: 0.0358, Accuracy: 9894/10000 (98.94%)

EPOCH:  14


Loss=0.07219089567661285 Batch_id=468 Accuracy = 99.11: 100%|██████████| 469/469 [00:08<00:00, 58.21it/s]  



Test set: Average loss: 0.0281, Accuracy: 9916/10000 (99.16%)

EPOCH:  15


Loss=0.0371481254696846 Batch_id=468 Accuracy = 99.19: 100%|██████████| 469/469 [00:07<00:00, 61.23it/s]   



Test set: Average loss: 0.0278, Accuracy: 9910/10000 (99.10%)



# Results & Analysis

**Results:**

Parameters: 7.8k

Best Train Accuracy: 99.19%

Best Test Accuracy: 99.16%

**Analysis:**

Over-fitting is reduced after adding dropout value & GAP layer
But 99.4 still cannot be reached even if model is pushed further

